In [31]:
from vrep_interface_library.vrepRobot import VREP_Environement, VREP_Robot
import vrep_interface_library.VREP_Python.vrep as vrep

import numpy as np
np.set_printoptions(precision=3, suppress=True)

import time

In [66]:
vrep_env = VREP_Environement(synchronous = True, dt = 0.05)

ct_robot_joint_handles = ['ct_robot_joint1', 
                    'ct_robot_joint2', 
                    'ct_robot_joint3', 
                    'ct_robot_joint4', 
                    'ct_robot_joint5', 
                    'ct_robot_joint6', 
                    'ct_robot_joint7', 
                    'ct_robot_joint8']

ct_robot_reference_frame_handles = ['reference_frame_j1', 
                    'reference_frame_j2',
                    'reference_frame_j3',
                    'reference_frame_j4',
                    'reference_frame_j5',
                    'reference_frame_j6',
                    'reference_frame_j7',
                    'reference_frame_j8',
                    'reference_frame_base']

vrep_reference_frame_handles = ['reference_frame_vrep_j1',
                           'reference_frame_vrep_j2',
                           'reference_frame_vrep_j3',
                           'reference_frame_vrep_j4',
                           'reference_frame_vrep_j5',
                           'reference_frame_vrep_j6',
                           'reference_frame_vrep_j7',
                           'reference_frame_vrep_j8',
                           'reference_frame_vrep_base']

dh_reference_frame_handles = ['reference_frame_dh_j1',
                           'reference_frame_dh_j2',
                           'reference_frame_dh_j3',
                           'reference_frame_dh_j4',
                           'reference_frame_dh_j5',
                           'reference_frame_dh_j6',
                           'reference_frame_dh_j7',
                           'reference_frame_dh_j8',
                           'reference_frame_dh_base']

vrep_env.add_robot(VREP_Robot('ct_robot_joints', ct_robot_joint_handles, connection_type = 'nonblocking'))
vrep_env.add_robot(VREP_Robot('ct_robot_reference_frames', ct_robot_reference_frame_handles, connection_type = 'nonblocking'))
vrep_env.add_robot(VREP_Robot('dh_reference_frames', dh_reference_frame_handles, connection_type = 'nonblocking'))
vrep_env.add_robot(VREP_Robot('vrep_reference_frames', vrep_reference_frame_handles, connection_type = 'nonblocking'))

Connected to remote API server
In synchronous mode


In [67]:
vrep_env.start_simulation()

vrep_env.ct_robot_joints.getCollisionHandle('Collision')
vrep.simxSynchronousTrigger(vrep_env.clientID)
vrep.simxGetPingTime(vrep_env.clientID)

vrep_env.ct_robot_joints.getCollisionState(initialize = True)
vrep.simxSynchronousTrigger(vrep_env.clientID)
vrep.simxGetPingTime(vrep_env.clientID)

for i in range(8):
    #setup joints for streaming gets
    vrep_env.ct_robot_joints.getJointPosition(vrep_env.ct_robot_joints.handles[i], initialize = True)

    #setup ct robot reference frames for streaming gets
    vrep_env.ct_robot_reference_frames.getObjectQuaternion(vrep_env.ct_robot_reference_frames.handles[i], relative2='reference_frame_base', initialize = True)
    vrep_env.ct_robot_reference_frames.getObjectPosition(vrep_env.ct_robot_reference_frames.handles[i], relative2='reference_frame_base', initialize = True)

    #setup dh reference frames for streaming gets
    vrep_env.dh_reference_frames.getObjectQuaternion(vrep_env.dh_reference_frames.handles[i], relative2='reference_frame_dh_base', initialize = True)
    vrep_env.dh_reference_frames.getObjectPosition(vrep_env.dh_reference_frames.handles[i], relative2='reference_frame_dh_base', initialize = True)    

    #setup vrep reference frames for streaming gets
    vrep_env.vrep_reference_frames.getObjectQuaternion(vrep_env.vrep_reference_frames.handles[i], relative2='reference_frame_vrep_base', initialize = True)
    vrep_env.vrep_reference_frames.getObjectPosition(vrep_env.vrep_reference_frames.handles[i], relative2='reference_frame_vrep_base', initialize = True)    
  
    #trigger
    vrep.simxSynchronousTrigger(vrep_env.clientID)
    vrep.simxGetPingTime(vrep_env.clientID)

4 robot(s) connected: ['ct_robot_joints', 'ct_robot_reference_frames', 'dh_reference_frames', 'vrep_reference_frames']


In [68]:
from kinematics_library.rigid_kinematics import *

pi = np.pi
robot = dh_robot_config(num_joints = 8, alpha = [-pi/2, pi/2, 0, pi/2, pi/2, -pi/2, -pi/2, 0], theta = [0, pi/2, 0, pi/2, pi/2, 0, -pi/2, 0], D = [0, 0, 0, 0, 0, 0, 0, 0.05], a = [0, 0, 0, 0, 0.08, 0.08, 0, 0], jointType = ['p', 'p', 'p', 'r', 'r', 'r', 'r', 'p'], ai = pi/2, aj = 0, ak = 0)

In [69]:
qs = np.array([0.5,0.2,0.2,0.,0.,0., 0.3, 0.0])

for i in range(8):
    
    #set joint positions
    for j in range(8):
        vrep_env.ct_robot_joints.setJointPosition(vrep_env.ct_robot_joints.handles[i], qs[i])
    
    vrep.simxSynchronousTrigger(vrep_env.clientID)
    vrep.simxGetPingTime(vrep_env.clientID)
    
    vrep_position = vrep_env.ct_robot_reference_frames.getObjectPosition(vrep_env.ct_robot_reference_frames.handles[i], relative2='reference_frame_base')[1]
    vrep_quaternion = vrep_env.ct_robot_reference_frames.getObjectQuaternion(vrep_env.ct_robot_reference_frames.handles[i], relative2='reference_frame_base')[1]
    vrep_quaternion = np.roll(vrep_quaternion, 1)
    
    #get my FK
    fk_position = robot.Tx(i, qs)
    fk_quat = robot.Orientation_quaternion(i,qs)
    
    #set dh vis coordinate frame positions / orientations
    vrep_env.dh_reference_frames.setObjectPosition(vrep_env.dh_reference_frames.handles[i], fk_position, relative2='reference_frame_dh_base')
    vrep_env.dh_reference_frames.setObjectQuaternion(vrep_env.dh_reference_frames.handles[i], np.roll(fk_quat,-1), relative2='reference_frame_dh_base') #roll due to (x,y,z,w) convention from VREP
    
    #set vrep vis coordinate frame positions / orientations
    vrep_env.vrep_reference_frames.setObjectPosition(vrep_env.vrep_reference_frames.handles[i], vrep_position, relative2='reference_frame_vrep_base')
    vrep_env.vrep_reference_frames.setObjectQuaternion(vrep_env.vrep_reference_frames.handles[i], np.roll(vrep_quaternion,-1), relative2='reference_frame_vrep_base') #roll due to (x,y,z,w) convention from VREP

    vrep.simxSynchronousTrigger(vrep_env.clientID)
    vrep.simxGetPingTime(vrep_env.clientID)
   
    
    
    print('---------------')
    print('VREP joint {} position: {}, quaternion: {}'.format(i+1, vrep_position, vrep_quaternion))
    print('FK joint {} position: {}, quaternion: {}'.format(i+1, fk_position, fk_quat))


#collisionState = vrep_env.ik_robot.getCollisionState()

#EEPosition[i,:] = vrep_env.ik_robot.getObjectPosition(vrep_env.ik_robot.handles[8], relative2='ik_rf7_static')[1]
#EEQuaternion[i,:] = vrep_env.ik_robot.getObjectQuaternion(vrep_env.ik_robot.handles[8], relative2='ik_rf7_static')[1]


---------------
VREP joint 1 position: [0.0, -0.20000001788139343, -5.960464477539063e-08], quaternion: [0.707 0.707 0.    0.   ]
FK joint 1 position: [ 0.  -0.5  0. ], quaternion: [0.707 0.707 0.    0.   ]
---------------
VREP joint 2 position: [0.0, -0.2000000774860382, 0.19999998807907104], quaternion: [ 0.707  0.    -0.     0.707]
FK joint 2 position: [ 0.  -0.5  0.2], quaternion: [0.707 0.    0.    0.707]
---------------
VREP joint 3 position: [0.20000001788139343, -0.2000000774860382, 0.19999998807907104], quaternion: [0.5 0.5 0.5 0.5]
FK joint 3 position: [ 0.2 -0.5  0.2], quaternion: [0.5 0.5 0.5 0.5]
---------------
VREP joint 4 position: [0.20000001788139343, -0.2000000774860382, 0.19999998807907104], quaternion: [-0.     0.707 -0.     0.707]
FK joint 4 position: [ 0.2 -0.5  0.2], quaternion: [ 0.    -0.707  0.    -0.707]
---------------
VREP joint 5 position: [0.20000001788139343, -0.2000000774860382, 0.19999998807907104], quaternion: [-0.707  0.707  0.    -0.   ]
FK joint 5

In [ ]:
vrep_env.shutdown()
